In [ ]:
from baseline_model import load_data_by_year
from baseline_model import drop_high_correlation_variables
from baseline_model import split_data_by_category
from baseline_model import drop_columns_from_data
from baseline_model import drop_columns_by_category_and_class
from baseline_model import load_aggregated_data
from baseline_model import evaluate_model_with_checks
from baseline_model import tune_model_with_random_search
from baseline_model import evaluate_model_performance_with_cv
from baseline_model import tune_logistic_regression
from baseline_model import evaluate_model_performance_with_cv_LR
from baseline_model import NeuralNet, compute_class_weight, train_model, evaluate_model, tune_neural_network
from baseline_model import evaluate_model_with_checks_NN, evaluate_model_performance_with_cv_NN

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
import joblib
import json
import os
import random

import sklearn.metrics
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV, ParameterGrid
from sklearn.metrics import roc_auc_score, precision_recall_curve, f1_score, precision_score, recall_score,make_scorer, average_precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold, mutual_info_classif
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

from xgboost import XGBClassifier
from sklearn.utils import resample
from scipy.stats import randint, uniform, loguniform

import torch
import torch.nn as nn
import torch.optim as optim
from imblearn.over_sampling import SMOTE
from torch.utils.data import DataLoader, TensorDataset



# Step1 Load data

In [ ]:
save_path = "./"
data_by_year_training, data_by_year_test = load_data_by_year(save_path)

In [ ]:
data_by_year_training, data_by_year_test = drop_high_correlation_variables(
    "high_correlation_variables.txt", data_by_year_training, data_by_year_test
)

In [7]:
categories = ['CKD', 'CKD_2', 'DIABETES', 'DIABETES_2', 'HYPERTENSION_ESSENTIAL', 
              'HYPERTENSION_ALL', 'HYPERTENSIVE_DISEASE', 'LIVER_DISEASE', 'CHRONIC_LIVER_DISEASE', 
              'CIRCULATORY_SYSTEM', 'ISCHEMIC_HEART_DISEASE', 'CONGESTIVE_HEART_FAILURE', 'STROKE', 
              'CARDIOVASCULAR_DISEASE', 'CANCER', 'COPD', 'PERIPHERAL_VASCULAR', 
              'NUTRITIONAL_DEFICIENCIES', 'METABOLIC_SYNDROME', 'SUBSTANCE_ABUSE']

split_data_by_category(categories, data_by_year_test, data_type="test")
split_data_by_category(categories, data_by_year_training, data_type="training")


In [8]:
columns_to_drop = ['CKD', 'CKD_2', 'DIABETES', 'DIABETES_2', 'HYPERTENSION_ESSENTIAL', 
              'HYPERTENSION_ALL', 'HYPERTENSIVE_DISEASE', 'LIVER_DISEASE', 'CHRONIC_LIVER_DISEASE', 
              'CIRCULATORY_SYSTEM', 'ISCHEMIC_HEART_DISEASE', 'CONGESTIVE_HEART_FAILURE', 'STROKE', 
              'CARDIOVASCULAR_DISEASE', 'CANCER', 'COPD', 'PERIPHERAL_VASCULAR', 
              'NUTRITIONAL_DEFICIENCIES', 'METABOLIC_SYNDROME', 'SUBSTANCE_ABUSE']
data_by_year_training = drop_columns_from_data(columns_to_drop, data_by_year_training)
data_by_year_test = drop_columns_from_data(columns_to_drop, data_by_year_test)

In [ ]:
categories = ['CKD', 'CKD_2', 'DIABETES', 'DIABETES_2', 'HYPERTENSION_ESSENTIAL', 
              'HYPERTENSION_ALL', 'HYPERTENSIVE_DISEASE', 'LIVER_DISEASE', 'CHRONIC_LIVER_DISEASE', 
              'CIRCULATORY_SYSTEM', 'ISCHEMIC_HEART_DISEASE', 'CONGESTIVE_HEART_FAILURE', 'STROKE', 
              'CARDIOVASCULAR_DISEASE', 'CANCER', 'COPD', 'PERIPHERAL_VASCULAR', 
              'NUTRITIONAL_DEFICIENCIES', 'METABOLIC_SYNDROME', 'SUBSTANCE_ABUSE']

columns_to_drop = ['CKD', 'CKD_2', 'DIABETES', 'DIABETES_2', 'HYPERTENSION_ESSENTIAL', 
              'HYPERTENSION_ALL', 'HYPERTENSIVE_DISEASE', 'LIVER_DISEASE', 'CHRONIC_LIVER_DISEASE', 
              'CIRCULATORY_SYSTEM', 'ISCHEMIC_HEART_DISEASE', 'CONGESTIVE_HEART_FAILURE', 'STROKE', 
              'CARDIOVASCULAR_DISEASE', 'CANCER', 'COPD', 'PERIPHERAL_VASCULAR', 
              'NUTRITIONAL_DEFICIENCIES', 'METABOLIC_SYNDROME', 'SUBSTANCE_ABUSE']

drop_columns_by_category_and_class(categories, columns_to_drop, data_type="test")

drop_columns_by_category_and_class(categories, columns_to_drop, data_type="training")

# Step2 train XG

In [ ]:
base_dir = "/"
os.makedirs(base_dir, exist_ok=True)

param_distributions = {
    'n_estimators': [100, 1000],  
    'max_depth': [5, 16],  
    'learning_rate': [0.01, 0.1],  
    'subsample': [0.8,0.9],  
    'colsample_bytree': [0.8,0.9]  
}

param_grid = list(ParameterGrid(param_distributions))
comb= len(param_grid)


all_hyperparameter_results = []

rolling_aggregated_metrics_results = {}
source_models_by_year = {}
categories = ['CKD', 'CKD_2', 'DIABETES', 'DIABETES_2', 'HYPERTENSION_ESSENTIAL', 
              'HYPERTENSION_ALL', 'HYPERTENSIVE_DISEASE', 'LIVER_DISEASE', 'CHRONIC_LIVER_DISEASE', 
              'CIRCULATORY_SYSTEM', 'ISCHEMIC_HEART_DISEASE', 'CONGESTIVE_HEART_FAILURE', 'STROKE', 
              'CARDIOVASCULAR_DISEASE', 'CANCER', 'COPD', 'PERIPHERAL_VASCULAR', 
              'NUTRITIONAL_DEFICIENCIES', 'METABOLIC_SYNDROME', 'SUBSTANCE_ABUSE']


for category in categories:
    for class_type in ['1', '0']:  
        variable_name = f"rolling_aggregated_metrics_results_{category}_{class_type}"
        
        globals()[variable_name] = {}

cutoff_year = 2011
data_by_year_totall = pd.concat(
    [data_by_year_training[2009], data_by_year_training[2010], data_by_year_training[2011]],
    axis=0
)

all_years = sorted(data_by_year_training.keys())
update_year=100
cutoff_year=2011

X_train, y_train = load_aggregated_data(
    data_by_year_totall=data_by_year_totall,
    data_by_year_training=data_by_year_training,
    cutoff_year=cutoff_year,
    update_year=update_year
)

best_model, best_params = tune_model_with_random_search(
    param_distributions=param_distributions,
    use_scale_pos_weight=True,  
    X_train=X_train,
    y_train=y_train,
    n_iter1=comb
)

model_path = os.path.join(base_dir, f"update_{update_year}_best_model_{cutoff_year}.pkl")
joblib.dump(best_model, model_path)
source_models_by_year[f"{cutoff_year}_update_{update_year}"] = model_path

rolling_aggregated_metrics_results[f"update_{update_year}_{cutoff_year}"] = {}


for category in categories:
    for class_type in ['1', '0']:
        variable_name = f"rolling_aggregated_metrics_results_{category}_{class_type}"
        results_dict = globals().get(variable_name, {})

        if results_dict is None:
            print(f"Variable {variable_name} not initialized.")
            continue

        key = f"update_{update_year}_{cutoff_year}"
        results_dict[key] = {}  

        print(f"  Updated {variable_name} with key={key}")

        globals()[variable_name] = results_dict





In [ ]:
results_storage = {}
for test_year in range(2011, 2022):  
    if test_year in data_by_year_test:  
        print(f"Testing year: {test_year}")
        
        metrics_mean = evaluate_model_performance_with_cv(
            data_by_year_test=data_by_year_test,
            test_year=test_year,
            best_model=best_model,
        )
        print(f"Metrics Mean: {metrics_mean}")
        
        if metrics_mean:  
            rolling_aggregated_metrics_results[f"update_{update_year}_{cutoff_year}"][test_year] = {
                'AUC': metrics_mean['AUC'],
                'PRAUC': metrics_mean['PRAUC'],
                'F1-Score': metrics_mean['F1'],
                'Precision': metrics_mean['Precision'],
                'Recall': metrics_mean['Recall']
            }
        else:
            print(f"Metrics for year {test_year} are invalid!")

    for category in categories:
        for class_type in ['1', '0']:
            test_data_variable = f"data_by_year_test_{category}_{class_type}"
            test_data_dict = globals().get(test_data_variable, {})
            
            if not test_data_dict or test_year not in test_data_dict:
                print(f"Missing test data for {category}_{class_type} in year {test_year}.")
                continue

            metrics_mean = evaluate_model_with_checks(
                test_data_dict=test_data_dict,
                test_year=test_year,
                best_model=best_model,
                category=category,
                class_type=class_type,
            )
            #print(f"Category: {category}, Class: {class_type}, Year: {test_year}, Metrics: {metrics_mean}")

            if category not in results_storage:
                results_storage[category] = {}
            if class_type not in results_storage[category]:
                results_storage[category][class_type] = {}
            if test_year not in results_storage[category][class_type]:
                results_storage[category][class_type][test_year] = {}

            if metrics_mean:  
                results_storage[category][class_type][test_year] = {
                    'AUC': metrics_mean.get('AUC', None),
                    'PRAUC': metrics_mean.get('PRAUC', None),
                    'F1-Score': metrics_mean.get('F1', None),
                    'Precision': metrics_mean.get('Precision', None),
                    'Recall': metrics_mean.get('Recall', None)
                }
            else:
                print(f"Invalid metrics for {category}_{class_type} in year {test_year}!")


In [ ]:
class_types = ['1', '0']


In [ ]:
base_dir = "./"
os.makedirs(base_dir, exist_ok=True) 

categories_class_types_path = os.path.join(base_dir, 'categories_and_class_types.json')
with open(categories_class_types_path, 'w') as f:
    json.dump({'categories': categories, 'class_types': class_types}, f)
print(f"Saved categories and class types to {categories_class_types_path}")

results_storage_path = os.path.join(base_dir, 'results_storage.json')
with open(results_storage_path, 'w') as f:
    json.dump(results_storage, f)
print(f"Saved results_storage to {results_storage_path}")

rolling_results_path = os.path.join(base_dir, 'rolling_aggregated_metrics_results.json')
with open(rolling_results_path, 'w') as f:
    json.dump(rolling_aggregated_metrics_results, f)
print(f"Saved rolling_aggregated_metrics_results to {rolling_results_path}")


# Step3 train LR

In [ ]:
base_dir_LR = "./"
os.makedirs(base_dir_LR, exist_ok=True)  

param_distributions_LR = {
    'C': [0.01, 0.1, 1, 10], 
    'penalty': ['l1'],  
    'solver': ['liblinear','saga'],  
    'max_iter': [5000],
    'tol': [1e-4],
}


param_grid_LR = list(ParameterGrid(param_distributions_LR))
comb_LR = len(param_grid_LR)

all_hyperparameter_results_LR = []

rolling_aggregated_metrics_results_LR = {}
source_models_by_year_LR = {}
categories = ['CKD', 'CKD_2', 'DIABETES', 'DIABETES_2', 'HYPERTENSION_ESSENTIAL', 
              'HYPERTENSION_ALL', 'HYPERTENSIVE_DISEASE', 'LIVER_DISEASE', 'CHRONIC_LIVER_DISEASE', 
              'CIRCULATORY_SYSTEM', 'ISCHEMIC_HEART_DISEASE', 'CONGESTIVE_HEART_FAILURE', 'STROKE', 
              'CARDIOVASCULAR_DISEASE', 'CANCER', 'COPD', 'PERIPHERAL_VASCULAR', 
              'NUTRITIONAL_DEFICIENCIES', 'METABOLIC_SYNDROME', 'SUBSTANCE_ABUSE']

for category in categories:
    for class_type in ['1', '0']:  
        variable_name_LR = f"rolling_aggregated_metrics_results_{category}_{class_type}_LR"
        
        globals()[variable_name_LR] = {}

cutoff_year_LR = 2011
data_by_year_totall_LR = pd.concat(
    [data_by_year_training[2009], data_by_year_training[2010], data_by_year_training[2011]],
    axis=0
)

all_years = sorted(data_by_year_training.keys())
update_year_LR = 100

cutoff_year_LR = 2011

X_train_LR, y_train_LR = load_aggregated_data(
    data_by_year_totall=data_by_year_totall_LR,
    data_by_year_training=data_by_year_training,
    cutoff_year=cutoff_year_LR,
    update_year=update_year_LR
)


best_model_LR, best_params_LR = tune_logistic_regression(
    param_distributions=param_distributions_LR,
    X_train=X_train_LR,
    y_train=y_train_LR,
    n_iter1=comb_LR
    
)

model_path_LR = os.path.join(base_dir_LR, f"update_{update_year_LR}_best_model_{cutoff_year_LR}.pkl")
joblib.dump(best_model_LR, model_path_LR)
source_models_by_year_LR[f"{cutoff_year_LR}_update_{update_year_LR}_LR"] = model_path_LR

rolling_aggregated_metrics_results_LR[f"update_{update_year_LR}_{cutoff_year_LR}"] = {}

for category in categories:
    for class_type in ['1', '0']:
        variable_name_LR = f"rolling_aggregated_metrics_results_{category}_{class_type}_LR"
        results_dict_LR = globals().get(variable_name_LR, {})

        if results_dict_LR is None:
            print(f"Variable {variable_name_LR} not initialized.")
            continue

        key_LR = f"update_{update_year_LR}_{cutoff_year_LR}"
        results_dict_LR[key_LR] = {}  
        print(f"  Updated {variable_name_LR} with key={key_LR}")

        globals()[variable_name_LR] = results_dict_LR

In [ ]:
results_storage_LR = {}

for test_year in range(2011, 2022):  
    if test_year in data_by_year_test:  
        print(f"Testing year: {test_year} for Logistic Regression")
        
        metrics_mean_LR = evaluate_model_performance_with_cv_LR(
            data_by_year_test=data_by_year_test,
            test_year=test_year,
            best_model_LR=best_model_LR,
        )
        print(f"Metrics Mean (LR): {metrics_mean_LR}")
        
        if metrics_mean_LR: 
            rolling_aggregated_metrics_results_LR[f"update_{update_year_LR}_{cutoff_year_LR}"][test_year] = {
                'AUC': metrics_mean_LR['AUC'],
                'PRAUC': metrics_mean_LR['PRAUC'],
                'F1-Score': metrics_mean_LR['F1'],
                'Precision': metrics_mean_LR['Precision'],
                'Recall': metrics_mean_LR['Recall']
            }
        else:
            print(f"Metrics for year {test_year} are invalid for Logistic Regression!")

    for category in categories:
        for class_type in ['1', '0']:
            test_data_variable_LR = f"data_by_year_test_{category}_{class_type}"
            test_data_dict_LR = globals().get(test_data_variable_LR, {})
            
            if not test_data_dict_LR or test_year not in test_data_dict_LR:
                print(f"Missing test data for {category}_{class_type} in year {test_year} (LR).")
                continue

            metrics_mean_LR = evaluate_model_with_checks(
                test_data_dict=test_data_dict_LR,
                test_year=test_year,
                best_model=best_model_LR,
                category=category,
                class_type=class_type,
            )

            if category not in results_storage_LR:
                results_storage_LR[category] = {}
            if class_type not in results_storage_LR[category]:
                results_storage_LR[category][class_type] = {}
            if test_year not in results_storage_LR[category][class_type]:
                results_storage_LR[category][class_type][test_year] = {}

            if metrics_mean_LR:  
                results_storage_LR[category][class_type][test_year] = {
                    'AUC': metrics_mean_LR.get('AUC', None),
                    'PRAUC': metrics_mean_LR.get('PRAUC', None),
                    'F1-Score': metrics_mean_LR.get('F1', None),
                    'Precision': metrics_mean_LR.get('Precision', None),
                    'Recall': metrics_mean_LR.get('Recall', None)
                }
            else:
                print(f"Invalid metrics for {category}_{class_type} in year {test_year} (LR)!")


In [ ]:
base_dir_LR = "./"
os.makedirs(base_dir_LR, exist_ok=True)  

categories_class_types_path_LR = os.path.join(base_dir_LR, 'categories_and_class_types_LR.json')
with open(categories_class_types_path_LR, 'w') as f:
    json.dump({'categories': categories, 'class_types': class_types}, f)
print(f"Saved categories and class types to {categories_class_types_path_LR}")

results_storage_path_LR = os.path.join(base_dir_LR, 'results_storage_LR.json')
with open(results_storage_path_LR, 'w') as f:
    json.dump(results_storage_LR, f)
print(f"Saved results_storage_LR to {results_storage_path_LR}")

rolling_results_path_LR = os.path.join(base_dir_LR, 'rolling_aggregated_metrics_results_LR.json')
with open(rolling_results_path_LR, 'w') as f:
    json.dump(rolling_aggregated_metrics_results_LR, f)
print(f"Saved rolling_aggregated_metrics_results_LR to {rolling_results_path_LR}")


# Step4 train NN

In [ ]:
base_dir_NN = "./"
os.makedirs(base_dir_NN, exist_ok=True)  

param_distributions = {
    'hidden_layer_sizes': [
    (512, 256, 128), (256, 128, 64), (128, 64, 32, 16), (512, 128), (256, 64)
    ],
    'activation': ['relu', 'tanh', 'leaky_relu'],  
    'alpha': [0.001, 0.1, 1000],  
    'learning_rate': [0.0001,0.001], 
    'max_iter': [500],  
    'batch_size': [64, 128, 256], 
}


cutoff_year_NN = 2011
data_by_year_totall_NN = pd.concat(
    [data_by_year_training[2009], data_by_year_training[2010], data_by_year_training[2011]],
    axis=0
)

all_years = sorted(data_by_year_training.keys())
update_year_NN = 100

X_train_NN, y_train_NN = load_aggregated_data(
    data_by_year_totall=data_by_year_totall_NN,
    data_by_year_training=data_by_year_training,
    cutoff_year=cutoff_year_NN,
    update_year=update_year_NN
)

X_train_NN = X_train_NN.astype(np.float32)  
y_train_NN = y_train_NN.astype(np.float32)

X_train_NN = X_train_NN.to_numpy()
y_train_NN = y_train_NN.to_numpy()

best_model_NN, best_params_NN = tune_neural_network(
    X_train=X_train_NN,
    y_train=y_train_NN,
    param_distributions=param_distributions,
    n_iter=270, 
    patience=30
)

model_path_NN = os.path.join(base_dir_NN, "best_nn_model.pth")
torch.save(best_model_NN.state_dict(), model_path_NN)

print(f"✅ Best model saved at {model_path_NN}")
print(f"Best hyperparameters: {best_params_NN}")




In [ ]:
rolling_aggregated_metrics_results_NN = {}

rolling_aggregated_metrics_results_NN[f"update_{update_year_NN}_{cutoff_year_NN}"] = {}

for category in categories:
    for class_type in ['1', '0']:
        variable_name_NN = f"rolling_aggregated_metrics_results_{category}_{class_type}_NN"
        results_dict_NN = globals().get(variable_name_NN, {})

        if results_dict_NN is None:
            print(f"Variable {variable_name_NN} not initialized.")
            continue

        key_NN = f"update_{update_year_NN}_{cutoff_year_NN}"
        results_dict_NN[key_NN] = {}  
        print(f"  Updated {variable_name_NN} with key={key_NN}")
        globals()[variable_name_NN] = results_dict_NN

In [ ]:
for test_year in range(2011, 2022):  
    if test_year in data_by_year_test: 
        print(f"Testing year: {test_year} for Neural Network")
        
        metrics_mean_NN = evaluate_model_performance_with_cv_NN(
            data_by_year_test=data_by_year_test,
            test_year=test_year,
            best_model_NN=best_model_NN,
        )
        print(f"Metrics Mean (NN): {metrics_mean_NN}")
        
        if metrics_mean_NN:  
            rolling_aggregated_metrics_results_NN[f"update_{update_year_NN}_{cutoff_year_NN}"][test_year] = {
                'AUC': metrics_mean_NN['AUC'],
                'PRAUC': metrics_mean_NN['PRAUC'],
                'F1-Score': metrics_mean_NN['F1'],
                'Precision': metrics_mean_NN['Precision'],
                'Recall': metrics_mean_NN['Recall']
            }
        else:
            print(f"Metrics for year {test_year} are invalid for Neural Network!")

In [ ]:
results_storage_NN = {}
for test_year in range(2011, 2022):  
    if test_year in data_by_year_test: 
        print(f"Testing year: {test_year}")

    for category in categories:
        for class_type in ['1', '0']:
            test_data_variable = f"data_by_year_test_{category}_{class_type}"
            test_data_dict = globals().get(test_data_variable, {})
            
            if not test_data_dict or test_year not in test_data_dict:
                print(f"Missing test data for {category}_{class_type} in year {test_year}.")
                continue

            metrics_mean = evaluate_model_with_checks_NN(
                test_data_dict=test_data_dict,
                test_year=test_year,
                best_model=best_model_NN,
                category=category,
                class_type=class_type,
            )
            #print(f"Category: {category}, Class: {class_type}, Year: {test_year}, Metrics: {metrics_mean}")

            if category not in results_storage_NN:
                results_storage_NN[category] = {}
            if class_type not in results_storage_NN[category]:
                results_storage_NN[category][class_type] = {}
            if test_year not in results_storage_NN[category][class_type]:
                results_storage_NN[category][class_type][test_year] = {}

            if metrics_mean:  
                results_storage_NN[category][class_type][test_year] = {
                    'AUC': metrics_mean.get('AUC', None),
                    'PRAUC': metrics_mean.get('PRAUC', None),
                    'F1-Score': metrics_mean.get('F1', None),
                    'Precision': metrics_mean.get('Precision', None),
                    'Recall': metrics_mean.get('Recall', None)
                }
            else:
                print(f"Invalid metrics for {category}_{class_type} in year {test_year}!")


In [ ]:
base_dir_NN = "./"
os.makedirs(base_dir_NN, exist_ok=True)  

categories_class_types_path_NN = os.path.join(base_dir_NN, 'categories_and_class_types_NN.json')
with open(categories_class_types_path_NN, 'w') as f:
    json.dump({'categories': categories, 'class_types': class_types}, f)
print(f"Saved categories and class types to {categories_class_types_path_NN}")

results_storage_path_NN = os.path.join(base_dir_NN, 'results_storage_NN.json')
with open(results_storage_path_NN, 'w') as f:
    json.dump(results_storage_NN, f)
print(f"Saved results_storage_NN to {results_storage_path_NN}")

rolling_results_path_NN = os.path.join(base_dir_NN, 'rolling_aggregated_metrics_results_NN.json')
with open(rolling_results_path_NN, 'w') as f:
    json.dump(rolling_aggregated_metrics_results_NN, f)
print(f"Saved rolling_aggregated_metrics_results_NN to {rolling_results_path_NN}")

